In [ ]:
# DATA QUALITY CHALLENGE -- REPORT -- SHINJINI SHOME
# EACH QUESTION USES A DIFFERENT CELL WHERE APPROACH AND FINDINGS ARE EXPLAINED IN COMMENTS

# Imports
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to SQLite database
conn = sqlite3.connect('challenge.db')

# Check all Tables for Duplicates using primary keys before proceeding

conv = "SELECT conv_id, user_id, conv_date, COUNT(*) FROM conversions GROUP BY conv_id, user_id, conv_date HAVING COUNT(*) > 1;"
print(pd.read_sql(conv,conn))

conv_back = "SELECT conv_id, user_id, conv_date, COUNT(*) FROM conversions_backend GROUP BY conv_id, user_id, conv_date HAVING COUNT(*) > 1;"
print(pd.read_sql(conv_back,conn))

session = "SELECT session_id, user_id, event_date, COUNT(*) FROM session_sources GROUP BY session_id, user_id, event_date HAVING COUNT(*) > 1;"
print(pd.read_sql(session,conn))

api_adword = "SELECT event_date, campaign_id, COUNT(*) FROM api_adwords_costs GROUP BY event_date, campaign_id HAVING COUNT(*) > 1;"
print(pd.read_sql(api_adword,conn))

cust_jour = "SELECT conv_id, session_id, COUNT(*) FROM attribution_customer_journey GROUP BY conv_id, session_id HAVING COUNT(*) > 1;"
print(pd.read_sql(cust_jour,conn))

# FINDINGS : NO DUPLICATES FOUND

In [ ]:
# Question 1: Are the costs in the 'api_adwords_costs' table fully covered in the 'session_sources' table? Any campaigns where you see issues?
# APPROACH :
# FINDINGS :

In [ ]:
# Question 2: Are the conversions in the 'conversions' table stable over time? Any pattern?

# APPROACH : Compare total revenue for each date and number of conversions for each date to view stability
# Also plot a scattter plot between number of conversions per day and total revenue to see any relationship or patterns between them.

# Get number of conversions and total revenue for each date
query_conv = "SELECT conv_date, COUNT(conv_id) AS Conversion_Count, SUM(revenue) AS Total_Revenue FROM conversions c GROUP BY conv_date ORDER BY conv_date;"
conv_revenue = pd.read_sql(query_conv,conn)
# print(conv_revenue)

# Plot the relationship of conversion counts and dates
plt.figure(figsize=(10, 6))
plt.plot(conv_revenue['conv_date'], conv_revenue['Conversion_Count'], marker='o')
plt.xlabel('Date')
plt.ylabel('Number of Conversions')
plt.title('Conversions Over Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Plot the relationship of total revenue and dates
plt.figure(figsize=(10, 6))
plt.plot(conv_revenue['conv_date'], conv_revenue['Total_Revenue'], marker='o')
plt.xlabel('Date')
plt.ylabel('Total Revenue')
plt.title('Revenue Over Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Plot the relationship of conversion counts with total revenue on a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(conv_revenue['Conversion_Count'], conv_revenue['Total_Revenue'], alpha=0.7)
plt.xlabel('Number of Conversions')
plt.ylabel('Total Revenue')
plt.title('Relationship between Conversions and Total Revenue')
plt.grid(True)
plt.show()

# FINDINGS :
# 1. Both number of conversions over time and revenue over time fluctuate.
# 2. Patterns are similar for the revenue and number of conversions, increase in number of conversions may increase total revenue.
# 3. Scatter plot confirms this somehow, except a few points where there is exception.

In [ ]:
# Question 3: Double check conversions ('conversions' table) with backend ('conversions_backend' table), any issues?

# APPROACH : Count the number of records in each table to see if they match, also do left and right joins to list all missing records or null value records.

query_c = "SELECT COUNT(*) FROM conversions;"
query_cb = "SELECT COUNT(*) FROM conversions_backend;"

query_leftj = "SELECT c.conv_id, c.user_id, c.conv_date FROM conversions c LEFT JOIN conversions_backend cb ON c.conv_id = cb.conv_id AND c.user_id = cb.user_id AND c.conv_date = cb.conv_date WHERE cb.conv_id IS NULL OR cb.user_id IS NULL OR cb.conv_date IS NULL;"
# Used left join as right join is not supported
query_rightj = "SELECT cb.conv_id, cb.user_id, cb.conv_date FROM conversions_backend cb LEFT JOIN conversions c ON c.conv_id = cb.conv_id AND c.user_id = cb.user_id AND c.conv_date = cb.conv_date WHERE c.conv_id IS NULL OR c.user_id IS NULL OR c.conv_date IS NULL;"

count_conv = pd.read_sql(query_c,conn)
count_convback = pd.read_sql(query_cb,conn)
missing_recs1 = pd.read_sql(query_leftj,conn)
missing_recs2 = pd.read_sql(query_rightj,conn)

print("Number of records in conversions table: " + str(count_conv['COUNT(*)'][0]))
print("\nNumber of records in conversions backend table: " + str(count_convback['COUNT(*)'][0]))

print("\nRecords present in conversions table that have null entries in conversions backend table: \n" + str(missing_recs1))
print("\nRecords present in conversions backend table that have null entries in conversions table: \n" + str(missing_recs2))

# FINDINGS :
# 1. Number of records don't match for conversions and conversions backend table.
# 2. The corresponding missing records for each table mapped to the other tables  is listed.

In [ ]:
# Question 4: Are attribution results consistent? Do you find any conversions where the 'ihc' values don't make sense?
# APPROACH :
# FINDINGS :

In [ ]:
# Question 5: (Bonus) Do we have an issue with channeling? Are the number of sessions per channel stable over time?

# APPROACH : Get number of sessions for each channel per day and plot line graphs to show stability over time for each.
# Study the unique channel names, seems to be issue with namings but lack of information so assumption is that they are unique.

# Get Channel and number of sessions per event_date 
query_channel = "SELECT  channel_name, event_date, COUNT(session_id) AS Number_of_Sessions FROM session_sources GROUP BY channel_name, event_date ORDER BY channel_name, event_date;"
channel_data = pd.read_sql(query_channel, conn)
# print(channel_data)

# Use of seaborn for style setting
sns.set(style="darkgrid")

# Get unique channel names
channels = channel_data['channel_name'].unique()

# Create subplots for each channel for clarity of visibility
fig, axes = plt.subplots(nrows=len(channels), ncols=1, figsize=(10, 5 * len(channels)))

for i, channel in enumerate(channels):
    # Filter data for the current channel
    single_channel_data = channel_data[channel_data['channel_name'] == channel]
    
    # Plot the line for the current channel
    sns.lineplot(data=single_channel_data, x='event_date', y='Number_of_Sessions', ax=axes[i], marker='o', markersize=8, label=channel)
    
    axes[i].set_title(f'Sessions Over Time - {channel}')
    axes[i].set_xlabel('Event Date')
    axes[i].set_ylabel('Session Count')
    axes[i].set_xticks(channel_data['event_date'])
    axes[i].tick_params(axis='x', rotation=45)
    
    # Display legend
    axes[i].legend(title='Channel', loc='upper right', bbox_to_anchor=(1, 1))

plt.tight_layout()
plt.show()

# FINDINGS : 
# 1. Issue with channeling : If channel names are duplicate - affiliate = affiliates, SEA brand = SEA branded, Social Paid = Social-Paid etc... this is unclear 
# 2. Most Channels not stable, the number of sessions fluctuate over time as seen in plots.


In [ ]:
# Question 6: (Bonus) Any other issues?
# APPROACH : TALK ABOUT SESSION INCONSISTENCIES
# FINDINGS :

# Close SQL connection
conn.close()